In [1]:
import numpy as np
import pandas as pd

In [5]:
train = pd.read_csv('train.csv',nrows = 1000)
train.shape

(1000, 3)

In [6]:
train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [7]:
#taking a look at sincere Questions
print("sincere questions")
train.loc[train['target']==0].sample(5)['question_text']

sincere questions


792    How is your experience with husky dogs in Indi...
294    Is laser a good option to remove these blavk l...
738    What does the chairman of a regional federal r...
532       What is the best diet for teddy bear hamsters?
788    What do you think will happen in Seasons 5 and...
Name: question_text, dtype: object

In [8]:
#print(insincere questions)
print('insincere questions')
train.loc[train['target']==1].sample(5)['question_text']

insincere questions


996    Why Chinese people are always not welcome in a...
615    Why do people on Quora not answer my question ...
460    How much more political fumbling will it take ...
803    Why are Europeans so ungrateful for America? T...
30     Which babies are more sweeter to their parents...
Name: question_text, dtype: object

In [9]:
samp = train.sample(1)
sentence = samp.iloc[0]['question_text']
print(sentence)

How do I find out the password for my Elite paycheck plus debit card if I haven’t set one up?


In [10]:
#preprocessing the text
import re
sentence = re.sub(r'\d+','',sentence)
print('Sentence after removing the numbers\n',sentence)

Sentence after removing the numbers
 How do I find out the password for my Elite paycheck plus debit card if I haven’t set one up?


In [11]:
#removing the punctuation marks
import string
translation_table = sentence.maketrans("","",string.punctuation)
translation_table

{33: None,
 34: None,
 35: None,
 36: None,
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,
 42: None,
 43: None,
 44: None,
 45: None,
 46: None,
 47: None,
 58: None,
 59: None,
 60: None,
 61: None,
 62: None,
 63: None,
 64: None,
 91: None,
 92: None,
 93: None,
 94: None,
 95: None,
 96: None,
 123: None,
 124: None,
 125: None,
 126: None}

In [12]:
sentence = sentence.translate(translation_table)
sentence

'How do I find out the password for my Elite paycheck plus debit card if I haven’t set one up'

In [13]:
#removing stopwords from the sentence
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
words_in_sentence = list(set(sentence.split())-stop_words)
print(words_in_sentence)

['one', 'card', 'find', 'debit', 'How', 'set', 'Elite', 'paycheck', 'plus', 'password', 'haven’t', 'I']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Prince\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Prince\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
#stemming of words
from nltk.stem import PorterStemmer
nltk.download('wordnet')
stemmer = PorterStemmer()
for i,word in enumerate(words_in_sentence):
    words_in_sentence[i]=stemmer.stem(word)
print(words_in_sentence)

['one', 'card', 'find', 'debit', 'how', 'set', 'elit', 'paycheck', 'plu', 'password', 'haven’t', 'i']


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Prince\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
#lemmatization of words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
words = []
for i,word in enumerate(words_in_sentence):
    words_in_sentence[i] = lemmatizer.lemmatize(word)
print(words_in_sentence)

['one', 'card', 'find', 'debit', 'how', 'set', 'elit', 'paycheck', 'plu', 'password', 'haven’t', 'i']


In [22]:
#splitting the train data
from sklearn.model_selection import train_test_split
train,test = train_test_split(train,test_size=0.2)

In [23]:
word_count={}
word_count_sincere={}
word_count_insincere={}
sincere=0
insincere= 0

import re
import string
import nltk
stop_words  =set(nltk.corpus.stopwords.words('english'))
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
row_count = train.shape[0]



for row in range(0,row_count):
    insincere += train.iloc[row]['target']
    sincere += (1-train.iloc[row]['target'])
    sentence = train.iloc[row]['question_text']
    sentence = re.sub(r'\d+',"",sentence)
    translation_table = sentence.maketrans("","",string.punctuation)
    sentence = sentence.translate(translation_table)
    words_in_sentence = list((set(sentence.split())-stop_words))
    for index,word in enumerate(words_in_sentence):
        word = stemmer.stem(word)
        words_in_sentence[index] =lemmatizer.lemmatize(word) 
    for word in words_in_sentence:
        if train.iloc[row]['target']==0:  #sincere words
            if word in word_count_sincere.keys():
                word_count_sincere[word]+=1
            else:
                word_count_sincere[word]=1
        elif train.iloc[row]['target']==1:  #insincere words
            if word in word_count_insincere.keys():
                word_count_insincere[word]+=1
            else:
                word_count_insincere[word]=1
        if word in word_count.keys():
            word_count[word]+=1
        else:
            word_count[word]=1
            
print('Done')

Done


- Finding probability for each word in the dictionary. After that eliminating words which are insignificant. Insignificant words are words which have a probability of occurence less than 0.0001.

In [24]:
word_probability = {}
total_words = 0
for i in word_count:
    total_words +=word_count[i]
for i in word_count:
    word_probability[i] = word_count[i]/total_words
    
#Eliminating the insignificant words
print('Total Words',len(word_probability))
print('Minimum probability',min(word_probability.values()))
threshold_p = 0.0001
for i in list(word_probability):
    if word_probability[i]<threshold_p:
        del word_prbability[i]
        if i in list(word_count_sincere):
            del word_count_sincere[i]
        elif i in list(word_count_insincere):
            del word_count_insincere[i]
print('Total Words',len(word_probability))

Total Words 2490
Minimum probability 0.0001691761123329386
Total Words 2490


- To apply naive bayes algorithm, we have to find conditional probability. Finding the conditional probability.

In [25]:
total_sincere_words = sum(word_count_sincere.values())
cp_sincere={}  #conditional probability
for i in list(word_count_sincere):
    cp_sincere[i] = word_count_sincere[i]/total_sincere_words
    
total_insincere_words =sum(word_count_insincere.values())
cp_insincere ={}
for i in list(word_count_insincere):
    cp_insincere[i] = word_count_insincere[i]/total_insincere_words

In [27]:
row_count=  test.shape[0]
p_sincere = sincere/(sincere+insincere)
p_insincere = insincere/(sincere+insincere)
accuracy = 0

for row in range(0,row_count):
    sentence  = test.iloc[row]['question_text']
    target = test.iloc[row]['target']
    sentence = re.sub(r'\d+','',sentence)
    sentence = sentence.translate(sentence.maketrans("","",string.punctuation))
    words_in_sentence = list(set(sentence.split())-stop_words)
    for index,word in enumerate(words_in_sentence):
        word = stemmer.stem(word)
        words_in_sentence[index] = lemmatizer.lemmatize(word)
    insincere_term = p_insincere
    sincere_term = p_sincere
    
    sincere_M = len(cp_sincere.keys())
    insincere_M = len(cp_insincere.keys())
    for word in words_in_sentence:
        if word not in cp_insincere.keys():
            insincere_M+=1
        if word not in cp_sincere.keys():
            sincere_M+=1
            
    for word in words_in_sentence:
        if word in cp_insincere.keys():
            insincere_term *= (cp_insincere[word] + (1/insincere_M))
        else:
            insincere_term *= (1/insincere_M)
        if word in cp_sincere.keys():
            sincere_term *= (cp_sincere[word] + (1/sincere_M))
        else:
            sincere_term *= (1/sincere_M)
        
    if insincere_term/(insincere_term + sincere_term) > 0.5:
        response = 1
    else:
        response = 0
    if target == response:
        accuracy += 1
    
print ('Accuracy is ',accuracy/row_count*100)
            

Accuracy is  12.0


- Here we have manually found out the accuracy using the calculations

### Naive Bayes Classifier

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [29]:
df = pd.read_csv('train.csv',nrows=2000)
df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [38]:
#creating the feature and target column
X = df['question_text']
y = df['target']

In [31]:
X.shape,y.shape

((2000,), (2000,))

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

# Sample documents
documents = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?",
]

# Create a CountVectorizer instance
vectorizer = CountVectorizer()

# Fit and transform the documents into a document-term matrix (DTM)
X = vectorizer.fit_transform(documents)

# # Get the vocabulary (unique words) and the DTM as a dense array
vocabulary = vectorizer.get_feature_names_out()
dtm_dense = X.toarray()

# # Print the results
print("Vocabulary:", vocabulary)
print("Document-Term Matrix (DTM):")
print(dtm_dense)

Vocabulary: ['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
Document-Term Matrix (DTM):
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


In [51]:
stemmer = PorterStemmer()
lammetizer = WordNetLemmatizer()
def pre_processing_text(sentence):
    sentence = re.sub(r'\d+',"",sentence) #remove numbers
    sentence  = sentence.translate(sentence.maketrans("","",string.punctuation)) #remove punctuation
    words = set(sentence.split())
    words = list(words - stop_words)
    words = ' '.join
    for index,word in enumerate(words):
        word = stemmer.stem(word)
        words[index]=lammetizer.lemmatize(word)
    return ' '.join(words)

In [44]:
X.head()

0    How did Quebec nationalists see their province...
1    Do you have an adopted dog, how would you enco...
2    Why does velocity affect time? Does velocity a...
3    How did Otto von Guericke used the Magdeburg h...
4    Can I convert montra helicon D to a mountain b...
Name: question_text, dtype: object

In [45]:
X[0]

'How did Quebec nationalists see their province as a nation in the 1960s?'

In [52]:
for i in range(X.shape[0]):
    X[i] = pre_processing_text(X[i])

C:\Users\Prince\AppData\Local\Temp\ipykernel_4728\905917026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = pre_processing_text(X[i])


In [53]:
X.head()

0            nationalist how quebec provinc nation see
1         adopt adopt peopl dog would shop encourag do
2             whi geometri veloc doe space time affect
3         magdeburg guerick how hemispher use otto von
4    can mountain helicon montra bike chang tyre co...
Name: question_text, dtype: object

In [58]:
X.shape

(2000,)

In [59]:
#Creating countvectorizer to convert the data into bag of words 
from sklearn.feature_extraction.text import CountVectorizer
 
vectorizer = CountVectorizer()
#splitting the data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
X_train.shape,y_train.shape

((1500,), (1500,))

In [ ]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [63]:
#Creating Multinomial NB model
NB = MultinomialNB()
NB.fit(X_train,y_train)

MultinomialNB()

In [66]:
#making predictions on y_test
y_pred= NB.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.936

- Model has 93.6 % accuracy

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
df = pd.read_csv('train.csv',nrows=2000)
#creating the feature and target column
X = df['question_text']
y = df['target']
stemmer = PorterStemmer()
lammetizer = WordNetLemmatizer()
def pre_processing_text(sentence):
    sentence = re.sub(r'\d+',"",sentence) #remove numbers
    sentence  = sentence.translate(sentence.maketrans("","",string.punctuation)) #remove punctuation
    words = set(sentence.split())
    words = list(words - stop_words)
    for index,word in enumerate(words):
        word = stemmer.stem(word)
        words[index]=lammetizer.lemmatize(word)
    return ' '.join(words)
for i in range(X.shape[0]):
    X[i] = pre_processing_text(X[i])
vectorizer = CountVectorizer()
#splitting the data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

#Creating Multinomial NB model
NB = MultinomialNB()
NB.fit(X_train,y_train)

#making predictions on y_test
y_pred= NB.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
accuracy

C:\Users\Prince\AppData\Local\Temp\ipykernel_4728\1007605482.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = pre_processing_text(X[i])


0.946

In [69]:
X

0               nationalist how quebec provinc nation see
1            adopt adopt peopl dog would shop encourag do
2                whi geometri veloc doe space time affect
3            magdeburg guerick how hemispher use otto von
4       can mountain helicon montra bike chang tyre co...
                              ...                        
1995                               what love two guy time
1996            what header benefit c file function defin
1997                         what rout line number bu brt
1998    left whi told peopl explicitli twice harri ron...
1999                        can ever nigeria nation great
Name: question_text, Length: 2000, dtype: object

In [71]:
X[0].split()

['nationalist', 'how', 'quebec', 'provinc', 'nation', 'see']